In [4]:
import pandas as pd
import deepctr

In [5]:
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names
from deepctr.models import * 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [6]:
# Sample data
data = {
    'user_id': [1, 2, 3, 4, 5],
    'item_id': [1, 2, 3, 4, 5],
    'gender': ['male', 'female', 'female', 'male', 'male'],
    'age': [23, 45, 35, 23, 30],
    'rating': [5, 4, 3, 2, 1]
}

df = pd.DataFrame(data)
df.head(2)

,user_id,item_id,gender,age,rating
0,1,1,male,23,5
1,2,2,female,45,4


In [7]:
sparse_features = ['user_id', 'item_id', 'gender']
dense_features = ['age']

In [8]:
fixlen_feature_columns = [
    deepctr.feature_column.SparseFeat(name, vocabulary_size=df[name].nunique(), embedding_dim=4) for name in sparse_features
] + [deepctr.feature_column.DenseFeat(name, 1) for name in dense_features]

fixlen_feature_columns[0]

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


SparseFeat(name='user_id', vocabulary_size=5, embedding_dim=4, use_hash=False, vocabulary_path=None, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.RandomNormal object at 0x000002ABA1350D88>, embedding_name='user_id', group_name='default_group', trainable=True)

In [9]:
feature_names = get_feature_names(fixlen_feature_columns)
feature_names 

['user_id', 'item_id', 'gender', 'age']

In [10]:
def print_feature_columns(feature_columns):
    for col in feature_columns:
        
        if isinstance(col, SparseFeat):
            print(f"SparseFeat - Name: {col.name}, Vocabulary Size: {col.vocabulary_size}, Embedding Dimension: {col.embedding_dim}")
        elif isinstance(col, DenseFeat):
            print(f"DenseFeat - Name: {col.name}, Dimension: {col.dimension}")
print_feature_columns(fixlen_feature_columns)

SparseFeat - Name: user_id, Vocabulary Size: 5, Embedding Dimension: 4
SparseFeat - Name: item_id, Vocabulary Size: 5, Embedding Dimension: 4
SparseFeat - Name: gender, Vocabulary Size: 2, Embedding Dimension: 4
DenseFeat - Name: age, Dimension: 1


In [11]:
for feature in sparse_features:
    lbe = LabelEncoder()
    df[feature] = lbe.fit_transform(df[feature])
df.head(2)

,user_id,item_id,gender,age,rating
0,0,0,1,23,5
1,1,1,0,45,4


In [12]:
model_input = {name: df[name] for name in feature_names}
model_input

{'user_id': 0    0
 1    1
 2    2
 3    3
 4    4
 Name: user_id, dtype: int64,
 'item_id': 0    0
 1    1
 2    2
 3    3
 4    4
 Name: item_id, dtype: int64,
 'gender': 0    1
 1    0
 2    0
 3    1
 4    1
 Name: gender, dtype: int32,
 'age': 0    23
 1    45
 2    35
 3    23
 4    30
 Name: age, dtype: int64}

In [13]:
train, test = train_test_split(df, test_size=0.2, random_state=42)
train, test

(   user_id  item_id  gender  age  rating
 4        4        4       1   30       1
 2        2        2       0   35       3
 0        0        0       1   23       5
 3        3        3       1   23       2,
    user_id  item_id  gender  age  rating
 1        1        1       0   45       4)

In [14]:
train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}
test_model_input

{'user_id': 1    1
 Name: user_id, dtype: int64,
 'item_id': 1    1
 Name: item_id, dtype: int64,
 'gender': 1    0
 Name: gender, dtype: int32,
 'age': 1    45
 Name: age, dtype: int64}

In [15]:
train_model_input

{'user_id': 4    4
 2    2
 0    0
 3    3
 Name: user_id, dtype: int64,
 'item_id': 4    4
 2    2
 0    0
 3    3
 Name: item_id, dtype: int64,
 'gender': 4    1
 2    0
 0    1
 3    1
 Name: gender, dtype: int32,
 'age': 4    30
 2    35
 0    23
 3    23
 Name: age, dtype: int64}

In [17]:
model = DeepFM(linear_feature_columns=fixlen_feature_columns,
               dnn_feature_columns=fixlen_feature_columns,
               task='regression')

make sure the activation function use training flag properly call() got an unexpected keyword argument 'training'


StagingError: in converted code:
    relative to C:\Users\OMEN:

    anaconda3\envs\deepctr\lib\site-packages\deepctr\layers\core.py:193 call *
        for i in range(len(self.hidden_units)):
    anaconda3\envs\deepctr\lib\site-packages\tensorflow\python\autograph\operators\control_flow.py:110 for_stmt
        return _py_for_stmt(iter_, extra_test, body, init_state)
    anaconda3\envs\deepctr\lib\site-packages\tensorflow\python\autograph\operators\control_flow.py:119 _py_for_stmt
        state = body(target, *state)
    AppData\Local\Temp\tmpqlk9v3_2.py:37 loop_body
        return deep_input_1, e_1

    UnboundLocalError: local variable 'e_1' referenced before assignment
